In [1]:
# Setup plot defaultrialsrialsrialss
import matplotlib.pyplot as plt
import seaborn as sns

# Reset styling to default
plt.rcdefaults()
# Grid on
plt.rcParams['axes.grid'] = True
# Top and right axis spines off
plt.rcParams['axes.spines.right'] = False
plt.rcParams['axes.spines.top'] = False

# Title 18 x y labels 16 ticks 14
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
fig_w=6
fig_h=6
plt.rcParams['figure.figsize'] = (fig_w,fig_h )
# Make axis equal
plt.rcParams['axes.axisbelow'] = True 


plt.rcParams['grid.color'] = '0.8'# Grid color

# Set default color cycle
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=['#043908', '#ff7f0e', '#9467bd', '#d62728', '#364704', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf'])
# Set default grid settings
plt.rcParams['grid.linestyle'] = '--'
plt.rcParams['grid.alpha'] = 0.7
plt.rcParams['font.family'] = 'sans-serif'

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

# psychometric function
# define the psychometric function
def psychometric_function(intensities, lapse_rate,mu, sigma):
    # Compute Φ(intensities; μ, σ) cdf is basically sum of probablity densities 
    # until certain point intensities is the x-axis values 
    cdf = norm.cdf(intensities, loc=mu, scale=sigma) 
    # cdf is the cumulative density function kind of like the integral of the 
    # pdf of the normal distribution so when we take one intenstiy lets say 0.6 it will give us the probability of the value being less than or equal to 0.6
    p_test = lapse_rate * 0.5 + (1 - lapse_rate) * cdf # p test at intensity
    return p_test

# number of successes on a given trial number and intensity
def simpsych(lambda_, mu, sigma, intensities, trials):
	intensities = np.array(intensities)
	trials = np.array(trials)
	success_per_intensity = []
	for i in range(len(intensities)):
		p_of_level = psychometric_function(intensities[i], lambda_, mu, sigma)
		success = 0
		for j in range(trials[i]):
			rand_decision = np.random.rand()
			if rand_decision < p_of_level:
				success += 1
		success_per_intensity.append(success)
	return np.array(success_per_intensity)

# Compute log-likelihoods for each candidate mu
def log_likelihood(p_I, B, T):
    # p_I is the probablity at given intensity
	# B is the number of successes PMF
	# T total number of trials
    # Add a small epsilon for numerical stability
    epsilon = 1e-9
    p_I = np.clip(p_I, epsilon, 1 - epsilon)
    return B 

# Negative log-likelihood function
def negative_log_likelihood(params, delta_dur, chose_test, total_responses):
    lambda_, mu, sigma = params
    p = psychometric_function(delta_dur, lambda_, mu, sigma)
    epsilon = 1e-9  # To prevent log(0)
    p = np.clip(p, epsilon, 1 - epsilon)
    log_likelihood = np.sum(chose_test * np.log(p) + (total_responses - chose_test) * np.log(1 - p))
    return -log_likelihood
